In [1]:
import numpy as np
import tqdm
from itertools import combinations
import random

In [4]:
SKB_dataset_name = "skb_da"
SKB_Benchmark_path = "../sememe_dataset/"

skb_data = np.load(SKB_Benchmark_path+"skb_da.npy", allow_pickle=True).tolist()
skb_da_cdv_map = np.load(SKB_Benchmark_path+"skb_da_cdv_map.npy", allow_pickle=True).tolist()

In [3]:
# for skb index
#def getSkbCDV(skb_data):
#    skb_da_cdv = set()
#    for word, sense in tqdm.tqdm(skb_data.items()):
#        for (_, sememe_set) in sense:
#            skb_da_cdv = skb_da_cdv | sememe_set
#    return {word:index for index, word in enumerate(skb_da_cdv)}

In [69]:
#skb_da_cdv_map = getSkbCDV(skb_data)

100%|██████████████████████████████████| 152661/152661 [11:00<00:00, 230.99it/s]


In [71]:
#np.save(SKB_Benchmark_path+"skb_da_cdv_map",skb_da_cdv_map)

In [5]:
def getRelationMap(skb_data):
    skb_data_map = {}
    skb_data_list = []
    triples_dict = {}
    for word, senses in tqdm.tqdm(skb_data.items()):
        for j, (pos, sems_set) in enumerate(senses):
            relations = word.replace(" ","_")+"||"+pos+"||"+str(j)
            skb_data_list.append(relations)
            triples_dict[relations] = sems_set
    skb_data_map = {word:index for index, word in enumerate(skb_data_list)}
    return skb_data_map,triples_dict

In [6]:
skb_data_map,triples_dict = getRelationMap(skb_data)

100%|████████████████████████████████| 152661/152661 [00:05<00:00, 25985.18it/s]


In [7]:
len(skb_data_map)

6312591

In [8]:
def getTriples(skb_data,triples_dict,skb_da_cdv_map,skb_data_map):
    triples_list = []
    for relation, sememes_set in tqdm.tqdm(triples_dict.items()):
        sememes_list = list(sememes_set)
        combinations_list = list(combinations(sememes_list, 2))
        if len(combinations_list) == 0:
            continue
        for (ea,eb) in combinations_list:
            triples_list.append((skb_da_cdv_map[ea],skb_da_cdv_map[eb],skb_data_map[relation]))
    return triples_list

In [9]:
triples_list = getTriples(skb_data,triples_dict,skb_da_cdv_map,skb_data_map)

100%|█████████████████████████████| 6312591/6312591 [00:51<00:00, 121626.44it/s]


In [10]:
len(triples_list)

152197006

In [11]:
#entity2id
def entity2Id(skb_da_cdv_map,write_path):
    skb_graph = open(write_path, 'w')
    for i, (sem, index) in enumerate(skb_da_cdv_map.items()):
        if i == 0:
            skb_graph.writelines(str(len(skb_da_cdv_map)))
            skb_graph.write('\n')
        skb_graph.writelines(sem+"\t"+str(index))
        skb_graph.write('\n')
    skb_graph.close()

In [12]:
#relation2id
def relation2Id(skb_data_map,write_path):
    skb_graph = open(write_path, 'w')
    for i, (sem, index) in enumerate(skb_data_map.items()):
        if i == 0:
            skb_graph.writelines(str(len(skb_data_map)))
            skb_graph.write('\n')
        skb_graph.writelines(sem+"\t"+str(index))
        skb_graph.write('\n')
    skb_graph.close()

In [13]:
random.shuffle(triples_list)

In [28]:
def writeTrainData(triples_list,write_path):
    skb_graph = open(write_path, 'w')
    for i, (e1, e2, ral) in enumerate(tqdm.tqdm(triples_list[:round(len(triples_list)*0.85)])):
        if i == 0:
            skb_graph.writelines(str(len(triples_list[:round(len(triples_list)*0.85)])))
            skb_graph.write('\n')
        skb_graph.writelines(str(e1)+"\t"+str(e2)+"\t"+str(ral))
        skb_graph.write('\n')
    skb_graph.close()
    
def writeTestData(triples_list,write_path):
    skb_graph = open(write_path, 'w')
    for i, (e1, e2, ral) in enumerate(tqdm.tqdm(triples_list[129367455:129367455+round(len(triples_list)*0.075)])):
        if i == 0:
            skb_graph.writelines(str(len(triples_list[129367455:129367455+round(len(triples_list)*0.075)])))
            skb_graph.write('\n')
        skb_graph.writelines(str(e1)+"\t"+str(e2)+"\t"+str(ral))
        skb_graph.write('\n')
    skb_graph.close()
    
def writeValidData(triples_list,write_path):
    skb_graph = open(write_path, 'w')
    for i, (e1, e2, ral) in enumerate(tqdm.tqdm(triples_list[129367455+11414775:])):
        if i == 0:
            skb_graph.writelines(str(len(triples_list[129367455+11414775:])))
            skb_graph.write('\n')
        skb_graph.writelines(str(e1)+"\t"+str(e2)+"\t"+str(ral))
        skb_graph.write('\n')
    skb_graph.close()

In [20]:
#train_data = triples_list[:round(len(triples_list)*0.85)]
#test_data = triples_list[len(train_data):len(train_data)+round(len(triples_list)*0.075)]
#valid_data = triples_list[len(train_data)+len(test_data):]

In [21]:
writeTrainData(triples_list,SKB_Benchmark_path+"TransE_4/train2id.txt")
writeTestData(triples_list,SKB_Benchmark_path+"TransE_4/test2id.txt:")
writeValidData(triples_list,SKB_Benchmark_path+"TransE_4/valid2id.txt")

100%|█████████████████████████| 129367455/129367455 [04:10<00:00, 516194.22it/s]


NameError: name 'train_data' is not defined

In [27]:
writeTestData(triples_list,SKB_Benchmark_path+"TransE_4/test2id.txt")

100%|███████████████████████████| 11414775/11414775 [00:21<00:00, 520120.22it/s]


In [29]:
writeValidData(triples_list,SKB_Benchmark_path+"TransE_4/valid2id.txt")

100%|███████████████████████████| 11414776/11414776 [00:21<00:00, 521872.17it/s]


In [30]:
entity2Id(skb_data_map,SKB_Benchmark_path+"TransE_4/entity2id.txt")

In [31]:
relation2Id(skb_data_map,SKB_Benchmark_path+"TransE_4/relation2id.txt")